<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_14_Notebook_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **Integração da API do Google com o LangChain**
---



In [ ]:
#@title Instalando a biblioteca de integração com o LangChain
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [ ]:
#@title Definindo a chave da API

import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API

from getpass import getpass

GOOGLE_API_KEY = getpass()

··········


**Integração**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

modelo = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = GOOGLE_API_KEY)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

mensagens = [
    SystemMessage(content="Traduza o seguinte do Inglês para o Português."),
    HumanMessage(content="hi!")
]

resposta = modelo.invoke(mensagens)

In [ ]:
resposta

AIMessage(content='Olá! \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-43f10b6a-0e56-4863-a334-0ea21b3b73c9-0', usage_metadata={'input_tokens': 14, 'output_tokens': 2, 'total_tokens': 16})

In [ ]:
resposta.content

'Oi!'

**ChatPromptTemplate**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Traduza o seguinte para {idioma}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{texto}")]
)

resultado = prompt_template.invoke({"idioma": "italiano", "texto": "hi"})

resultado

ChatPromptValue(messages=[SystemMessage(content='Traduza o seguinte para italiano:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [ ]:
resultado.to_messages()

[SystemMessage(content='Traduza o seguinte para italiano:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [ ]:
chain =  prompt_template | modelo

resposta = chain.invoke({"idioma": "português", "texto": "hi"})

In [ ]:
resposta

AIMessage(content='Oi', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--4874a149-2cc9-4f18-8087-cf3903062b42-0', usage_metadata={'input_tokens': 10, 'output_tokens': 26, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 25}})

In [ ]:
resposta.content

'Oi'

**Encadeando componentes com LCEL**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
chain = prompt_template | modelo | parser

In [ ]:
chain.invoke({"idioma": "portugues", "texto": "hi"})

'Olá'

**Cadeias**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = GOOGLE_API_KEY)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Me conte uma piada sobre {assunto}")
chain = prompt | chat

In [ ]:
resposta = chain.invoke({"assunto": "papagaio"})

In [ ]:
from IPython.display import Markdown

Markdown(resposta.content)

Claro, aqui vai uma:

Um homem compra um papagaio que, segundo o vendedor, era super falador.

Ele leva o papagaio para casa, tenta de tudo: fala com ele, canta para ele, mas o papagaio fica em silêncio absoluto por dias.

Frustrado, o homem se aproxima da gaiola e desabafa:
— Poxa, você não vai falar nunca?!

O papagaio vira a cabeça, olha bem para ele e, com uma voz idêntica à do vendedor, responde:
— É que na casa do antigo dono ele falava tanto que me venderam!

Mais um exemplo:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

modelo = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = GOOGLE_API_KEY)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente que traduz {idioma_input} para {idioma_output}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | modelo

resposta = chain.invoke(
    {
        "idioma_input": "Inglês",
        "idioma_output": "Português",
        "input": "I love programming.",
    }
)

In [ ]:
resposta

AIMessage(content='Eu amo programar.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--2bfa3984-4dc7-427e-8bc2-07f0fd1159ee-0', usage_metadata={'input_tokens': 20, 'output_tokens': 36, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 31}})

In [ ]:
resposta.content

'Eu amo programar.'